# Question B2 (10 marks)
In Question B1, we used the Category Embedding model. This creates a feedforward neural network in which the categorical features get learnable embeddings. In this question, we will make use of a library called Pytorch-WideDeep. This library makes it easy to work with multimodal deep-learning problems combining images, text, and tables. We will just be utilizing the deeptabular component of this library through the TabMlp network:

In [3]:
!pip install pytorch-widedeep

  Using cached imutils-0.5.4-py3-none-any.whl
  Using cached pydantic-2.7.0-py3-none-any.whl.metadata (103 kB)
  Using cached annotated_types-0.6.0-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/21.9 MB ? eta -:--:--
   -- ------------------------------------- 1.4/21.9 MB 30.5 MB/s eta 0:00:01
   --- ------------------------------------ 2.1/21.9 MB 19.1 MB/s eta 0:00:02
   ---- ----------------------------------- 2.6/21.9 MB 15.3 MB/s eta 0:00:02
   ----- ---------------------------------- 3.1/21.9 MB 13.3 MB/s eta 0:00:02
   ------ --------------------------------- 3.7/21.9 MB 13.3 MB/s eta 0:00:02
   ------- -------------------------------- 3.9/21.9 MB 11.8 MB/s eta 0:00:02
   ------- -------------------------------- 4.1/21.9 MB 11.0 MB/s eta 0:00:02
   -------- ------------------------------- 4.6/21.9 MB 11.0 MB/s eta 0:00:02
   --------- ------------------------------ 5.2/21.9 MB 11.1 MB/s eta 0:00:02
   ---------- ----------------------------- 5.

  You can safely remove it manually.


In [4]:
!pip install numpy==1.21

   ---------------------------------------- 0.0/14.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/14.0 MB 363.1 kB/s eta 0:00:39
    --------------------------------------- 0.3/14.0 MB 1.3 MB/s eta 0:00:11
   - -------------------------------------- 0.7/14.0 MB 2.9 MB/s eta 0:00:05
   - -------------------------------------- 0.7/14.0 MB 2.9 MB/s eta 0:00:05
   - -------------------------------------- 0.7/14.0 MB 2.9 MB/s eta 0:00:05
   --- ------------------------------------ 1.2/14.0 MB 3.0 MB/s eta 0:00:05
   ------ --------------------------------- 2.2/14.0 MB 5.1 MB/s eta 0:00:03
   ------ --------------------------------- 2.2/14.0 MB 5.1 MB/s eta 0:00:03
   ------ --------------------------------- 2.2/14.0 MB 5.1 MB/s eta 0:00:03
   -------- -------------

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.58.1 requires numpy<1.27,>=1.22, but you have numpy 1.21.0 which is incompatible.
pytorch-widedeep 1.5.1 requires numpy>=1.21.6, but you have numpy 1.21.0 which is incompatible.
tensorflow-intel 2.13.0 requires numpy<=1.24.3,>=1.22, but you have numpy 1.21.0 which is incompatible.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.10.0 which is incompatible.


In [14]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

from pytorch_widedeep.preprocessing import TabPreprocessor
from pytorch_widedeep.models import TabMlp, WideDeep
from pytorch_widedeep import Trainer
from pytorch_widedeep.metrics import R2Score

import torch
torch.manual_seed(SEED)

In [8]:
def set_seed(SEED=42):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)

1.Divide the dataset (‘hdb_price_prediction.csv’) into train and test sets by using entries from the year 2020 and before as training data, and entries from 2021 and after as the test data.

In [12]:
df = pd.read_csv('hdb_price_prediction.csv')

# YOUR CODE HERE
# Divide dataset into training and test sets
train_df = df[df['year'] <= 2020]
test_df = df[df['year'] >= 2021]

# Sanity Check: Get unique values 
train_unique_years = ', '.join(map(str, train_df['year'].unique()))
test_unique_years = ', '.join(map(str, test_df['year'].unique()))

# Sanity Check: Print the formatted unique values
print("Unique years in train_df: " + train_unique_years)
print(f"Unique years in test_df: {test_unique_years}")

SyntaxError: invalid syntax (2639165518.py, line 13)

2.Refer to the documentation of Pytorch-WideDeep and perform the following tasks:
https://pytorch-widedeep.readthedocs.io/en/latest/index.html
* Use [**TabPreprocessor**](https://pytorch-widedeep.readthedocs.io/en/latest/examples/01_preprocessors_and_utils.html#2-tabpreprocessor) to create the deeptabular component using the continuous
features and the categorical features. Use this component to transform the training dataset.
* Create the [**TabMlp**](https://pytorch-widedeep.readthedocs.io/en/latest/pytorch-widedeep/model_components.html#pytorch_widedeep.models.tabular.mlp.tab_mlp.TabMlp) model with 2 linear layers in the MLP, with 200 and 100 neurons respectively.
* Create a [**Trainer**](https://pytorch-widedeep.readthedocs.io/en/latest/pytorch-widedeep/trainer.html#pytorch_widedeep.training.Trainer) for the training of the created TabMlp model with the root mean squared error (RMSE) cost function. Train the model for 100 epochs using this trainer, keeping a batch size of 64. (Note: set the *num_workers* parameter to 0.)

In [11]:
# YOUR CODE & RESULT HERE
# Utilize Pytorch-WideDeep Library

set_seed()

# TabPreprocessor: Preprocess the data 
# Define the 'column set up'
cat_embed_cols = [
    "month",
    "town",
    "flat_model_type",
    "storey_range",
]
continuous_cols = [
    "dist_to_nearest_stn",
    "dist_to_dhoby",
    "degree_centrality",
    "eigenvector_centrality",
    "remaining_lease_years",
    "floor_area_sqm",
]

# Prepare the data
tab_preprocessor = TabPreprocessor(cat_embed_cols=cat_embed_cols, 
                                    continuous_cols=continuous_cols, 
                                    cols_to_scale=continuous_cols)
X_tab_train = tab_preprocessor.fit_transform(train_df)
X_tab_test = tab_preprocessor.transform(test_df)

# TabMlp: Build the TabMlp model
tab_mlp = TabMlp(
    column_idx=tab_preprocessor.column_idx,
    cat_embed_input=tab_preprocessor.cat_embed_input,
    continuous_cols=continuous_cols,
    mlp_hidden_dims=[200, 100]
)

# Create the WideDeep model with TabMlp as the deeptabular component
model = WideDeep(deeptabular=tab_mlp)

# Trainer: Train the model
# Getting the target values
target_train = train_df['resale_price'].values
target_test = test_df['resale_price'].values

# Train and validate
trainer = Trainer(model=model, objective="rmse", num_workers=0, seed=SEED)
trainer.fit(
    X_tab=X_tab_train,
    target=target_train,
    n_epochs=100,
    batch_size=64, 
    val_split=0.3, # 70-30 split
)

# Train DF Shape
# (87370, 14)
# 70-30 Split means 61159 rows for training and 26211 rows for validation
# Batch size of 64 further means 956 batches for training and 410 batches for validation

valid: 100%|██████████████████████████████████████████████████████████| 410/410 [00:04<00:00, 101.68it/s, loss=3.84e+4]


valid: 100%|███████████████████████████████████████████████████████████| 410/410 [00:04<00:00, 96.58it/s, loss=3.71e+4]


3.Report the test RMSE and the test R2 value that you obtained.

In [19]:
from sklearn.metrics import mean_squared_error, r2_score

# TODO: Enter your code here

set_seed()
# make predictions on the test set
preds = trainer.predict(X_tab=X_tab_test).ravel()

# Test DF Shape
# (72183, 14)
# 1128 Iterations because 72183 rows / 64 batch size = 1128

# Calculate the Root Mean Squared Error (RMSE) between the predicted and actual target values
rmse = np.sqrt(mean_squared_error(target_test, preds))

# Calculate the R-squared (R²) score, which measures the goodness of fit of the model
r2 = r2_score(target_test, preds)

# Print the RMSE and R² scores to evaluate the model's performance on the test dataset
print()
print('-------------------------------')
print('Model Performance on Test Set:')
print(f"Test RMSE: {round(rmse,4)} (4 d.p.)")
print(f"Test R²  : {round(r2,4)}      (4 d.p.)")
print('-------------------------------')

predict: 100%|████████████████████████████████████████████████████████████████████| 1128/1128 [00:05<00:00, 196.75it/s]


-------------------------------
Model Performance on Test Set:
Test RMSE: 116158.649 (4 d.p.)
Test R²  : 0.5286      (4 d.p.)
-------------------------------
